# Notes

- Careful for time slots, verify everything lines up (triple check) don't want a repeat of what happened last time Johnny boy
- Also, are there missing events? Verify that all events are accounted for...
- - I am noticing some dropoff of events... Check rows and make sure we are all accounted for. For example original tables

In [66]:
# Run when refreshing data!!!!!!!
# %run WebScraping.py # Refreshes all the original tables

## Last updated (Tables 0, 1, 2)
# Friday, May 23rd, 2025
#   Last Updated: Wed, 05/14/2025, 2:28 pm
# Saturday, May 24th, 2025
#   Last Updated: Thu, 05/15/2025, 6:00 pm
# Sunday, May 25th, 2025
#   Last Updated: Tue, 05/13/2025, 7:45 pm

import pandas as pd
import ast  # convert string list

#Run when needing to overwrite any new events or whatnot
categories_JRW = pd.read_csv("Event_Categories_JRW.csv").drop(columns={'Color','Data Value'})
categories_JRW['Category'] = categories_JRW['Category'].str.strip() # clean merge

for ii in range(0, 3):
    df_final = pd.read_csv(f"AnimeBoston_day{ii}_schedule.csv", converters={'TimeSlot': ast.literal_eval})
    df_final['Category'] = df_final['Category'].str.strip() # clean merge
    df_merge = pd.merge(df_final, categories_JRW, on='Category', how='left')
    df_merge.to_csv(f"AnimeBoston_day{ii}_schedule_JRW.csv", index=False, encoding='utf-8-sig')

dfs = []
for ii in range(3):# loop over days
    # read each file
    df = pd.read_csv(f'AnimeBoston_day{ii}_schedule_JRW.csv')
    dfs.append(df)

# combine all days
df_all = pd.concat(dfs, ignore_index=True)

# filter NaN Utility
df_nan_utility = df_all[df_all['Utility_general_JRW'].isna()]
df_nan_utility

,Event,Room,Category,Color,TimeSlot,Utility_general_JRW


,Event,Room,Category,Color,TimeSlot,Utility_general_JRW


,Event,Room,Category,Color,TimeSlot,Utility


In [62]:
df_nan_utility['Category'].to_list()

[]

### What's next

Tidying
- Need to have a way to link back and make sure that the events exploded list is consistent... could just explode this list

Scoring needed!
- Have a way to score this, probably write to csv, then score utility from 1 - 10, 10 being amazing, and 1 being absolutely not, this could be rated from multiple people and averaged among them (or take the minimum among all there)

Would be nice, best practice, can be later
- Afterward, there is a checking process for seeing unique constraints on the time slots, this could be a function or some such



In [37]:
df_final_exploded = df_final.explode('TimeSlot').sort_values(by=['Room', 'Event']).reset_index(drop=True)
df_final_exploded

,Event,Room,Category,Color,TimeSlot
0,Charity Ball,"('Hynes', 'Ballroom A')",Social Event,#F7931E,16:30
1,Charity Ball,"('Hynes', 'Ballroom A')",Social Event,#F7931E,16:45
2,Charity Ball,"('Hynes', 'Ballroom A')",Social Event,#F7931E,17:00
3,Charity Ball,"('Hynes', 'Ballroom A')",Social Event,#F7931E,17:15
4,Charity Ball,"('Hynes', 'Ballroom A')",Social Event,#F7931E,17:30
...,...,...,...,...,...
533,Sketchbook Swap,"('Sheraton', 'Workshop Fairfax')",Social Event|Workshop,#F7931E,17:45
534,Sketchbook Swap,"('Sheraton', 'Workshop Fairfax')",Social Event|Workshop,#F7931E,18:00
535,Sketchbook Swap,"('Sheraton', 'Workshop Fairfax')",Social Event|Workshop,#F7931E,18:15
536,Sketchbook Swap,"('Sheraton', 'Workshop Fairfax')",Social Event|Workshop,#F7931E,18:30


In [38]:
df_final_timeslotgrouped = df_final_exploded.groupby(['TimeSlot'])[['Event']].agg(set).reset_index()
df_final_timeslotgrouped

,TimeSlot,Event
0,00:00,"{Slumber Party, Japan and the World's Game: Ex..."
1,00:15,"{Slumber Party, Japan and the World's Game: Ex..."
2,00:30,{Slumber Party}
3,00:45,{Slumber Party}
4,01:00,"{Slumber Party, Intro to J-horror}"
...,...,...
67,22:45,"{Fandom Obasan- 25 Years of BL fandom, SportsB..."
68,23:00,"{Connecting the Borders: A ClariS Fan Panel, S..."
69,23:15,"{Connecting the Borders: A ClariS Fan Panel, S..."
70,23:30,"{Connecting the Borders: A ClariS Fan Panel, J..."


In [39]:
# keep only first duplicates
# make Event lists hashable for deduplication
df_final_timeslotgrouped["Event_tuple"] = df_final_timeslotgrouped["Event"].apply(tuple)

# drop duplicates based on the Event list only, keeping the first time slot
df_dedupped_by_event = df_final_timeslotgrouped.drop_duplicates(subset="Event_tuple", keep="first")

# drop the helper column
df_dedupped_by_event = df_dedupped_by_event.drop(columns="Event_tuple").reset_index()
df_dedupped_by_event


,index,TimeSlot,Event
0,0,00:00,"{Slumber Party, Japan and the World's Game: Ex..."
1,2,00:30,{Slumber Party}
2,4,01:00,"{Slumber Party, Intro to J-horror}"
3,6,01:30,{Intro to J-horror}
4,8,08:00,{Samurai Troopers Ronin Warriors}
5,12,09:00,{From Screen to Kitchen: Adapting Existing Wor...
6,14,09:30,{From Screen to Kitchen: Adapting Existing Wor...
7,16,10:00,"{Opening Ceremonies, The History of Digimon in..."
8,18,10:30,"{Opening Ceremonies, The History of Digimon in..."
9,20,11:00,"{Opening Ceremonies, Yassify Your Cosplay Make..."


In [108]:
category_utility_base = {
    '18+ Cosplay': 7,
    '18+ Event': 6,
    '18+ Fan Creations': 6,
    '18+ Fan Panel': 5,
    '18+ Featured Panel': 6,
    '18+ Gameshow': 7,
    '18+ Guest Panel': 6,
    '18+ Video': 7,
    '21+ Event': 5,
    'Ball': 0, # assumed
    'Concerts': 7,
    'Cosplay Games': 6,
    'Event': 5,
    'Fan Creations': 6,
    'Fan Panel': 5,
    'Featured Artist': 5,
    'Featured Panel': 5,
    'Gameshow': 6,
    'Guest Panel': 5,
    'ID Check': 1,
    'Idol Events': 5,
    'Industry Panel': 5,
    'Jam Zone': 6,
    'Karaoke': 5,
    'Libraries & Education': 5,
    'Lolita & J-fashion': 5,
    'Maid Cafe': 7,
    'Premiere Video': 6,
    'Social Gatherings': 5,
    'Tabletop Gaming': 4,
    'Video': 4,
    'Workshop': 5,
}
category_utility_base

{'18+ Cosplay': 7,
 '18+ Event': 6,
 '18+ Fan Creations': 6,
 '18+ Fan Panel': 5,
 '18+ Featured Panel': 6,
 '18+ Gameshow': 7,
 '18+ Guest Panel': 6,
 '18+ Video': 7,
 '21+ Event': 5,
 'Ball': 0,
 'Concerts': 7,
 'Cosplay Games': 6,
 'Event': 5,
 'Fan Creations': 6,
 'Fan Panel': 5,
 'Featured Artist': 5,
 'Featured Panel': 5,
 'Gameshow': 6,
 'Guest Panel': 5,
 'ID Check': 1,
 'Idol Events': 5,
 'Industry Panel': 5,
 'Jam Zone': 6,
 'Karaoke': 5,
 'Libraries & Education': 5,
 'Lolita & J-fashion': 5,
 'Maid Cafe': 7,
 'Premiere Video': 6,
 'Social Gatherings': 5,
 'Tabletop Gaming': 4,
 'Video': 4,
 'Workshop': 5}